## Imports


In [3]:
%load_ext autoreload
%autoreload 2

#Imports
import geopandas as gpd
import numpy as np
import pandas as pd

#k-means
import data_weighted_kmeans
import importlib

#graph
import folium
import random

#widget
import ipywidgets as widgets
import os
import time
import selenium
from selenium import webdriver


## Manage Data 

In [6]:
# Load data
iris = gpd.read_file("data/CONTOURS-IRIS_2-1__SHP__FRA_2017-06-30/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2016/CONTOURS-IRIS_2-1_SHP_LAMB93_FE-2016/CONTOURS-IRIS.shp")

# Change projection
iris = iris.to_crs(epsg='4326')
iris.head()

population_2014 = pd.read_excel("data/population_iris/base-ic-evol-struct-pop-2014.xls", skiprows=4, header=1)


In [7]:
N=population_2014["P14_POP"].sum()
n=population_2014.groupby("DEP")["P14_POP"].sum()
N
h = 335*n/N
p=h.astype('int')

In [10]:
#distribute circonscription by departement
df_rep = pd.DataFrame()
df_rep['nb_circo'] = h.values
df_rep['population']=n.values
df_rep['nb_circo_int'] = p.values
df_rep['nb_circo_int'] = df_rep['nb_circo_int'].replace(0,1)
df_rep['dep']=population_2014.groupby("DEP")["DEP"]
rest = int(df_rep['nb_circo'].sum()-df_rep['nb_circo_int'].sum())+1
df_rep['nb_circo_reste'] = df_rep['nb_circo'] - df_rep['nb_circo_int']
df_rep = df_rep.sort_values('nb_circo_reste', ascending=False)
df_rep['nb_circo_reste_arrondi'] = (df_rep['nb_circo_reste'] >= df_rep['nb_circo_reste'].nlargest(n=rest).min()).astype('int') 
df_rep['circo_total'] = df_rep['nb_circo_int'] + df_rep['nb_circo_reste_arrondi'] 
df_rep = df_rep.sort_values('dep')
#get rid of corsica, north and dom-tom
df_final = df_rep.drop(df_rep.index[[28,29,95,96,97,98,99]]).copy()
df_final['population']=(df_final['population']/df_final['circo_total']).astype('int')
df_final.index = range(len(df_final.index))
df_final


,nb_circo,population,nb_circo_int,dep,nb_circo_reste,nb_circo_reste_arrondi,circo_total
0,3.182546,208709,3,"(01, [01, 01, 01, 01, 01, 01, 01, 01, 01, 01, ...",0.182546,0,3
1,2.743667,179927,2,"(02, [02, 02, 02, 02, 02, 02, 02, 02, 02, 02, ...",0.743667,1,3
2,1.743752,171531,1,"(03, [03, 03, 03, 03, 03, 03, 03, 03, 03, 03, ...",0.743752,1,2
3,0.821337,161587,1,"(04, [04, 04, 04, 04, 04, 04, 04, 04, 04, 04, ...",-0.178663,0,1
4,0.711012,139883,1,"(05, [05, 05, 05, 05, 05, 05, 05, 05, 05, 05, ...",-0.288988,0,1
5,5.506375,216662,5,"(06, [06, 06, 06, 06, 06, 06, 06, 06, 06, 06, ...",0.506375,0,5
6,1.638633,161190,1,"(07, [07, 07, 07, 07, 07, 07, 07, 07, 07, 07, ...",0.638633,1,2
7,1.421765,279715,1,"(08, [08, 08, 08, 08, 08, 08, 08, 08, 08, 08, ...",0.421765,0,1
8,0.775520,152573,1,"(09, [09, 09, 09, 09, 09, 09, 09, 09, 09, 09, ...",-0.224480,0,1
9,1.566013,154047,1,"(10, [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, ...",0.566013,1,2


In [12]:
iris_filtered= {}

for i in range(1,95):
    if i<10 :
        departement = "0"+str(i)
        
    else :
        departement = str(i)
    if i!=20:    
        iris_filtered[i] = iris[iris["CODE_IRIS"].str.startswith(departement)].copy()
        
    
population_2014["CODE_IRIS"] = population_2014["IRIS"]


dept = 1

for key, value in iris_filtered.items():
         
        value.loc[:,'P14_POP'] = population_2014['P14_POP']
        value.loc[:, 'departement_iris'] = dept
        dept = dept + 1
        value.loc[:, 'centroid_lng'] = value["geometry"].centroid.apply(lambda x: x.x)
        value.loc[:, 'centroid_lat'] = value["geometry"].centroid.apply(lambda x: x.y)
iris_filtered

{1:       INSEE_COM                      NOM_COM  IRIS  CODE_IRIS  \
 12        01027                        Balan  0000  010270000   
 86        01082                       Chanay  0000  010820000   
 488       01424                     Tramoyes  0000  014240000   
 659       01241                   Meillonnas  0000  012410000   
 725       01404            Serrières-sur-Ain  0000  014040000   
 1175      01242                     Mérignat  0000  012420000   
 1373      01311                   Prémillieu  0000  013110000   
 1710      01210                        Lélex  0000  012100000   
 1732      01112                    Condamine  0000  011120000   
 1773      01153                    Échenevex  0000  011530000   
 1891      01261                    Monthieux  0000  012610000   
 1893      01207                   Lapeyrouse  0000  012070000   
 1932      01036            Belmont-Luthézieu  0000  010360000   
 1956      01097                    Chavornay  0000  010970000   
 1969  

## Create & save map

In [24]:
%%time
importlib.reload(data_weighted_kmeans)
mapa = folium.Map([46.575859, 0.290518],
              zoom_start=9,
              tiles='cartodbpositron')
map_filtered={}
k=320
count=1;
#for k in range(300,600,25):
for key, value in iris_filtered.items():
    #get rid of corsica and north (problem with map generation)
    if key != 59 and key != 20:
        points = []
        if key<20:
            nb= df_final['circo_total'][key-1]
            pop = df_final['population'][key-1]
        if key>20:
            nb= df_final['circo_total'][key-2]
            pop = df_final['population'][key-2]
       
        
        for idx, row in value.iterrows():
            points.append({"coords": np.array([float(row['centroid_lng']), float(row['centroid_lat'])]), \
                          "w": int(row["P14_POP"]), "zip": row['CODE_IRIS'], "state": row["departement_iris"]})
        centers = data_weighted_kmeans.randomize_initial_cluster(points, nb)

        points, centers, it_num = data_weighted_kmeans.kmeans_evolution_weighted(points, centers, nb, it_max=1000, weight_step_scale=10)
        points_df = pd.DataFrame.from_dict(points)
        points_df["CODE_IRIS"] = points_df["zip"]
        points_df["coords"] = "aaa"
        result = value.merge(points_df, how='inner', on=['CODE_IRIS', 'CODE_IRIS'])
        simplified_map = result.dissolve(by='c')
        simplified_map["colour"] = ["#%06x" % random.randint(0, 0xFFFFFF) for i in range(0,nb)]
        points = folium.features.GeoJson(simplified_map,  style_function=lambda feature: { 'fillColor': feature['properties']['colour'],
            'color' : "#000000",
            'weight' : 1,
            'fillOpacity' : 0.5,
            }
        )
        map_filtered["map{0}".format(key)] = points
        mapa.add_children(points)
        
        for i in range(0,len(centers)):
            center = centers[i]
            folium.Marker([center["coords"][1],center["coords"][0]],
                            popup="population :"+str(pop),
                            icon=folium.Icon(color='red', icon='info-sign')).add_to(mapa)

fn='map_'+str(k)+'circo.html'
mapa.save(fn)
    

C:\Users\LHOUMEAU\Anaconda3_\lib\site-packages\ipykernel_launcher.py:39: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.


Wall time: 3min 45s
